# 2018年9月全球科技前沿报告
------

国务院发展研究中心国际技术经济研究所

数据部 康宸

数据采集自“**全球技术地图**”

2018.10.25

In [97]:
from IPython.display import display_html
display_html("""
<div style="text-align:center; margin: 20px 0;">
<p style="color:#FA5882;text-align:center; margin: 10px 0 20px 0;">
如果您希望隐藏代码，只显示数据来源、分析过程与结论。<br/>
请点击后面的“隐藏代码”按钮。
</p>
<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">隐藏代码</button>
<hr/>
</div>
""", 
             raw=True
            )

如果您希望隐藏代码，只显示数据来源、分析过程与结论。 
请点击后面的“隐藏代码”按钮。
 
 隐藏代码

In [98]:
import numpy as np
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
import imageio
import jieba
import jieba.analyse
import sys
import re
from pyecharts import Bar,Pie,Style,Map,HeatMap,WordCloud
import echarts_cities_pypkg,echarts_countries_pypkg,echarts_themes_pypkg
jieba.analyse.set_idf_path("./dicts/idf.txt")
jieba.load_userdict('./dicts/userdict.txt')

from pyecharts import online
online()


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [99]:
def get_from_text(pd_all, field_name):
    '''
    提取对应领域文本，用jieba分词形成list
    :param pd_all:
        包含所有内容的DataFrame
    :param field_name:
        领域名称str
    '''
    d = pd_all
    text = "".join(d[d.field==field_name]['content_clean'])
    seg_list = jieba.lcut(text, cut_all=False)
    return seg_list

def word_count(word_lst):
    word_dict = {}
    for i in word_lst:
        if i not in word_dict: 
            word_dict[i] = 1
        else:
            word_dict[i] += 1
    return word_dict

def load_stopdict(f):
        '''
        Load personalized dict to improve detect rate.

        Parameter:
            - f : A plain text file contains words and their ocurrences.
                  Can be a file-like object, or the path of the dictionary file,
                  whose encoding must be utf-8.

        Structure of dict file:
        word1 freq1 word_type1
        word2 freq2 word_type2
        ...
        Word type may be ignored
        '''
        stopwords = []
        re_userdict = re.compile('^(.+?)( [0-9]+)?( [a-z]+)?$', re.U)
        f_name = f
        f = open(f, 'rb')
        for lineno, ln in enumerate(f, 1):
            line = ln.strip()
            line = line.decode('utf-8').lstrip('\ufeff')
            word, freq, tag = re_userdict.match(line).groups()
            if freq is not None:
                freq = freq.strip()
            if tag is not None:
                tag = tag.strip()
            stopwords.append(word)
        return stopwords

stop_words = load_stopdict('./dicts/stop_words.txt')

In [100]:
sep_d = pd.read_csv('./1809/sep_all_nat.csv',sep='|',header=0)
sep_d.describe()

,title,content,field,source,date,content_clean,area
count,320,320,320,320,320,320,320
unique,320,320,9,124,28,320,63
top,美能源部宣布为碳捕集转换技术投入3000万美元,据Phys.org网站9月5日消息，日本北海道大学研究人员开发出一种新型催化剂，可在650摄...,能源,国防科技信息网,9月17日,日本原子能机构宣布，其成功开发出用于检查福岛第一核电站放射性污染图像的技术。该成像技术将虚拟...,美国
freq,1,1,60,29,22,1,151


## 数据概览

2018年9月1日——2018年9月30日

从**124**个信息源，


采集**9**个领域，


**63**个国家或地区，


**320**件科技前沿动态

-----

In [101]:
field_stat = pd.DataFrame(sep_d['field'].value_counts())
field_stat

,field
能源,60
信息,56
生物,47
科技战略,32
海洋,31
新材料,24
航空,24
航天,23
先进制造,23


In [102]:
plt.rcParams['font.family'] = ['Source Han Sans CN']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [103]:
field_stat['proportion'] = field_stat['field'].map(lambda x: x/320)
field_stat

attr = [i for i in field_stat.index]
value = [i for i in field_stat.proportion]
pie = Pie("2018年9月全球科技各领域事件占比", title_pos='left', width=1000, height=600, title_text_size=28, background_color='#404a59')
pie.use_theme('dark')
pie.add("领域", attr, value, center=[40, 50], is_random=True, radius=[20, 75], rosetype='area',
        is_legend_show=False, is_label_show=True)

pie

In [104]:
source_stat = pd.DataFrame(sep_d['source'].value_counts())

style = Style(
        title_top=10,
        title_color="#fff",
        title_pos="left",
        title_text_size=28,
        subtitle_text_size=15,
        width=1000,
        height=700,
        background_color='#404a59'
    )

bar = Bar("2018年9月全球科技前沿动态信息源统计",' ',  **style.init_style)
bar.use_theme('dark')
bar.add('科技动态', 
        source_stat.index,
        source_stat.source,
        xaxis_interval=0, 
        xaxis_rotate=30,
        is_label_show=True, 
        is_datazoom_show=True,
        datazoom_type="both",
        datazoom_range=[0, 15],
        is_legend_show=False,
        label_color=['rgba(255,30,0,100)']
       )

bar

In [105]:
date_stat = pd.DataFrame(sep_d['date'].value_counts())

data_cal = []
for i in date_stat.index:
    timeStruct = time.strptime(i, "%m月%d日")
    j = time.strftime("2018-%m-%d", timeStruct)
    data_cal.append([j, date_stat.date[i]])

style = Style(
        title_color="#fff",
        title_text_size=28,
        subtitle_text_size=15,
        width=980,
        height=600,
        background_color='#404a59'
    )

heatmap = HeatMap("2018年9月全球科技活动时间热力图", "每日科技前沿动态", **style.init_style)
heatmap.use_theme('dark')
heatmap.add(
    "",
    data_cal,
    is_calendar_heatmap=True,
    visual_text_color="#fff",
    visual_text_size=20,
    visual_range_text=["不活跃", "活跃"],
    visual_range=[0, 22],
    calendar_cell_size=[100, 65],
    is_visualmap=True,
    calendar_date_range="2018-09",
    visual_orient="vertical",
    visual_pos="right"
)
heatmap

In [106]:
import getnation.need_list as need_list
nation_dict = need_list.nation_dict

In [107]:
sep_us = sep_d[sep_d['area'].str.contains(nation_dict['us']['pattern'])]
sep_cn = sep_d[sep_d['area'].str.contains(nation_dict['cn']['pattern'])]
sep_eu = sep_d[sep_d['area'].str.contains(nation_dict['eu']['pattern'])]
sep_uk = sep_d[sep_d['area'].str.contains(nation_dict['uk']['pattern'])]
sep_fr = sep_d[sep_d['area'].str.contains(nation_dict['fr']['pattern'])]
sep_ge = sep_d[sep_d['area'].str.contains(nation_dict['ge']['pattern'])]
sep_ca = sep_d[sep_d['area'].str.contains(nation_dict['ca']['pattern'])]
sep_au = sep_d[sep_d['area'].str.contains(nation_dict['au']['pattern'])]
sep_jp = sep_d[sep_d['area'].str.contains(nation_dict['jp']['pattern'])]
sep_kr = sep_d[sep_d['area'].str.contains(nation_dict['kr']['pattern'])]
sep_sg = sep_d[sep_d['area'].str.contains(nation_dict['sg']['pattern'])]
sep_ru = sep_d[sep_d['area'].str.contains(nation_dict['ru']['pattern'])]
sep_id = sep_d[sep_d['area'].str.contains(nation_dict['id']['pattern'])]
sep_is = sep_d[sep_d['area'].str.contains(nation_dict['is']['pattern'])]
sep_ch = sep_d[sep_d['area'].str.contains(nation_dict['ch']['pattern'])]
sep_se = sep_d[sep_d['area'].str.contains(nation_dict['se']['pattern'])]
sep_dk = sep_d[sep_d['area'].str.contains(nation_dict['dk']['pattern'])]
sep_nl = sep_d[sep_d['area'].str.contains(nation_dict['nl']['pattern'])]
sep_fi = sep_d[sep_d['area'].str.contains(nation_dict['fi']['pattern'])]
sep_pl = sep_d[sep_d['area'].str.contains(nation_dict['pl']['pattern'])]
sep_es = sep_d[sep_d['area'].str.contains(nation_dict['es']['pattern'])]
sep_at = sep_d[sep_d['area'].str.contains(nation_dict['at']['pattern'])]
sep_it = sep_d[sep_d['area'].str.contains(nation_dict['it']['pattern'])]
sep_qa = sep_d[sep_d['area'].str.contains(nation_dict['qa']['pattern'])]
sep_ir = sep_d[sep_d['area'].str.contains(nation_dict['ir']['pattern'])]
sep_sa = sep_d[sep_d['area'].str.contains(nation_dict['sa']['pattern'])]
sep_in = sep_d[sep_d['area'].str.contains(nation_dict['in']['pattern'])]
sep_br = sep_d[sep_d['area'].str.contains(nation_dict['br']['pattern'])]

In [108]:
data_all = [
    ('United States', sep_us['title'].count()),
    ('China', sep_cn['title'].count()),
    ('United Kingdom', sep_uk['title'].count()),
    ('France', sep_fr['title'].count()),
    ('Germany', sep_ge['title'].count()),
    ('Canada', sep_ca['title'].count()),
    ('Australia', sep_au['title'].count()),
    ('Japan', sep_jp['title'].count()),
    ('Korea', sep_kr['title'].count()),
    ('Singapore', sep_sg['title'].count()),
    ('Russia', sep_ru['title'].count()),
    ('India', sep_id['title'].count()),
    ('Israel', sep_is['title'].count()),
    ('Switzerland', sep_ch['title'].count()),
    ('Sweden', sep_se['title'].count()),
    ('Denmark', sep_dk['title'].count()),
    ('Netherlands', sep_nl['title'].count()),
    ('Finland', sep_fi['title'].count()),
    ('Poland', sep_pl['title'].count()),
    ('Spain', sep_es['title'].count()),
    ('Austria', sep_at['title'].count()),
    ('Italy', sep_it['title'].count()),
    ('Qatar', sep_qa['title'].count()),
    ('Iran', sep_ir['title'].count()),
    ('Saudi Arabia', sep_sa['title'].count()),
    ('Brazil', sep_br['title'].count())
]

style = Style(
        title_top=30,
        title_color="#fff",
        title_pos="center",
        title_text_size=28,
        subtitle_text_size=15,
        width=1000,
        height=600,
        background_color='#404a59'
    )
chart = Map("2018年9月全球科技领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_all)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,
              visual_range=[0, 30], visual_text_color='#fff',visual_range_text=['不活跃', '活跃'],
              is_legend_show=False, is_map_symbol_show=False)
chart

# 各领域分析

## 科技战略

2018年9月共**32条**动态

In [109]:
def get_field_src(pd_all, field_name):
    '''
    提取对应领域文本，用jieba分词形成list
    :param pd_all:
        包含所有内容的DataFrame
    :param field_name:
        领域名称str
    '''
    d = pd_all
    src = (d[d.field == field_name]['source'])

    src_stat = pd.DataFrame(src.value_counts())
    style = Style(
        title_top=10,
        title_color="#fff",
        title_pos="left",
        title_text_size=28,
        width=980,
        height=600,
        background_color='#404a59'
    )

    bar = Bar("2018年9月" + field_name + "信息源统计", "data from 全球技术地图",  **style.init_style)
    bar.use_theme('dark')
    bar.add('科技动态', 
        src_stat.index, 
        src_stat.source,
        xaxis_interval=0, 
        xaxis_rotate=30,
        is_label_show=True, 
        is_datazoom_show=True,
        datazoom_type="both",
        datazoom_range=[0, 30],
        is_legend_show=False,
        label_color=['rgba(255,30,0,100)']
           )
    return bar

bar = get_field_src(sep_d, '科技战略')
bar

In [110]:
def get_field_date(pd_all, field_name, v_range):
    d = pd_all
    date = (d[d.field == field_name]['date'])
    date_stat = pd.DataFrame(date.value_counts())
    data_cal = []
    for i in date_stat.index:
        timeStruct = time.strptime(i, "%m月%d日")
        j = time.strftime("2018-%m-%d", timeStruct)
        data_cal.append([j, date_stat.date[i]])

    style = Style(
        title_color="#fff",
        title_text_size=28,
        subtitle_text_size=15,
        width=1000,
        height=600,
        background_color='#404a59'
    )

    heatmap = HeatMap("2018年9月全球" + field_name + "时间热力图",
                      "每日科技前沿动态", **style.init_style)
    heatmap.add(
        "",
        data_cal,
        is_calendar_heatmap=True,
        visual_text_color="#fff",
        visual_text_size=20,
        visual_range_text=["不活跃", "活跃"],
        visual_range=v_range,
        calendar_cell_size=[120, 65],
        is_visualmap=True,
        calendar_date_range="2018-09",
        visual_orient="horizontal",
        visual_pos="left",
        item_textcolor="#fff"
    )
    return heatmap
heatmap = get_field_date(sep_d, '科技战略', [1, 5])
heatmap

In [111]:
data_kj = [
    ('United States', sep_us[sep_us.field=='科技战略']['title'].count()),
    ('China', sep_cn[sep_cn.field=='科技战略']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='科技战略']['title'].count()),
    ('France', sep_fr[sep_fr.field=='科技战略']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='科技战略']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='科技战略']['title'].count()),
    ('Australia', sep_au[sep_au.field=='科技战略']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='科技战略']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='科技战略']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='科技战略']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='科技战略']['title'].count()),
    ('India', sep_id[sep_id.field=='科技战略']['title'].count()),
    ('Israel', sep_is[sep_is.field=='科技战略']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='科技战略']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='科技战略']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='科技战略']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='科技战略']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='科技战略']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='科技战略']['title'].count()),
    ('Spain', sep_es[sep_es.field=='科技战略']['title'].count()),
    ('Austria', sep_at[sep_at.field=='科技战略']['title'].count()),
    ('Italy', sep_it[sep_it.field=='科技战略']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='科技战略']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='科技战略']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='科技战略']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='科技战略']['title'].count())
]

chart = Map("2018年9月全球科技战略活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_kj)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 20], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [112]:
def get_wc(fx,nub=None,shapewc="diamond"):
    list = get_from_text(sep_d, fx)
    word_count_dict = word_count(list)
    data_wc = []
    for k, v in word_count_dict.items():
        if (k not in stop_words) & (v > 5):
            data_wc.append((k, v))
    for k, v in word_count_dict.items():
        if (k not in stop_words) & (v < 6) & (v > 2):
            data_wc.append((k, v))
    for k, v in word_count_dict.items():
        if (k not in stop_words) & (v == 2):
            data_wc.append((k, v))
    style_wc = Style(
        title_top=10,
        title_color="#000",
        title_pos="left",
        title_text_size=30,
        subtitle_text_size=15,
        width=1000,
        height=700,
        background_color='#fff'
    )

    chart = WordCloud("2018年9月" + fx + "热门关键词",
                      "data from 全球技术地图", **style_wc.init_style)
    name, value = chart.cast(data_wc[:nub])
    chart.add("",
              name,
              value,
              shape=shapewc,
              word_gap=30,
              word_size_range=[20, 100])
    return chart

chart = get_wc('科技战略', nub = 100)
chart

## 信息

2018年9月共**56条**动态


In [113]:
bar = get_field_src(sep_d, '信息')
bar

In [114]:
heatmap = get_field_date(sep_d, '信息', [1, 5])
heatmap

In [115]:
data_xx = [
    ('United States', sep_us[sep_us.field=='信息']['title'].count()),
    ('China', sep_cn[sep_cn.field=='信息']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='信息']['title'].count()),
    ('France', sep_fr[sep_fr.field=='信息']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='信息']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='信息']['title'].count()),
    ('Australia', sep_au[sep_au.field=='信息']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='信息']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='信息']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='信息']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='信息']['title'].count()),
    ('India', sep_id[sep_id.field=='信息']['title'].count()),
    ('Israel', sep_is[sep_is.field=='信息']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='信息']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='信息']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='信息']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='信息']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='信息']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='信息']['title'].count()),
    ('Spain', sep_es[sep_es.field=='信息']['title'].count()),
    ('Austria', sep_at[sep_at.field=='信息']['title'].count()),
    ('Italy', sep_it[sep_it.field=='信息']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='信息']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='信息']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='信息']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='信息']['title'].count())
]

chart = Map("2018年9月全球信息领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_xx)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 5], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [116]:
chart = get_wc('信息',nub=150)
chart

## 生物

2018年9月共**47条**动态


In [117]:
bar = get_field_src(sep_d, '生物')
bar

In [118]:
heatmap = get_field_date(sep_d, '生物', [1, 5])
heatmap

In [119]:
data_sw = [
    ('United States', sep_us[sep_us.field=='生物']['title'].count()),
    ('China', sep_cn[sep_cn.field=='生物']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='生物']['title'].count()),
    ('France', sep_fr[sep_fr.field=='生物']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='生物']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='生物']['title'].count()),
    ('Australia', sep_au[sep_au.field=='生物']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='生物']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='生物']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='生物']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='生物']['title'].count()),
    ('India', sep_id[sep_id.field=='生物']['title'].count()),
    ('Israel', sep_is[sep_is.field=='生物']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='生物']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='生物']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='生物']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='生物']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='生物']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='生物']['title'].count()),
    ('Spain', sep_es[sep_es.field=='生物']['title'].count()),
    ('Austria', sep_at[sep_at.field=='生物']['title'].count()),
    ('Italy', sep_it[sep_it.field=='生物']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='生物']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='生物']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='生物']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='生物']['title'].count())
]

chart = Map("2018年9月全球生物领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_sw)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 5], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [120]:
chart = get_wc('生物', nub=150)
chart

## 能源

2018年9月共**60条**动态

In [121]:
bar = get_field_src(sep_d, '能源')
bar

In [122]:
heatmap = get_field_date(sep_d, '能源', [1, 10])
heatmap

In [123]:
data_ny = [
    ('United States', sep_us[sep_us.field=='能源']['title'].count()),
    ('China', sep_cn[sep_cn.field=='能源']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='能源']['title'].count()),
    ('France', sep_fr[sep_fr.field=='能源']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='能源']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='能源']['title'].count()),
    ('Australia', sep_au[sep_au.field=='能源']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='能源']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='能源']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='能源']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='能源']['title'].count()),
    ('India', sep_id[sep_id.field=='能源']['title'].count()),
    ('Israel', sep_is[sep_is.field=='能源']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='能源']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='能源']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='能源']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='能源']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='能源']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='能源']['title'].count()),
    ('Spain', sep_es[sep_es.field=='能源']['title'].count()),
    ('Austria', sep_at[sep_at.field=='能源']['title'].count()),
    ('Italy', sep_it[sep_it.field=='能源']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='能源']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='能源']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='能源']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='能源']['title'].count())
]

chart = Map("2018年9月全球能源领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_ny)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 10], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [124]:
chart = get_wc('能源', nub=120, shapewc='pentagon')
chart

## 海洋

2018年9月共**31条**动态

In [125]:
bar = get_field_src(sep_d, '海洋')
bar

In [126]:
heatmap = get_field_date(sep_d, '海洋', [1, 3])
heatmap

In [127]:
data_hy = [
    ('United States', sep_us[sep_us.field=='海洋']['title'].count()),
    ('China', sep_cn[sep_cn.field=='海洋']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='海洋']['title'].count()),
    ('France', sep_fr[sep_fr.field=='海洋']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='海洋']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='海洋']['title'].count()),
    ('Australia', sep_au[sep_au.field=='海洋']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='海洋']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='海洋']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='海洋']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='海洋']['title'].count()),
    ('India', sep_id[sep_id.field=='海洋']['title'].count()),
    ('Israel', sep_is[sep_is.field=='海洋']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='海洋']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='海洋']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='海洋']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='海洋']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='海洋']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='海洋']['title'].count()),
    ('Spain', sep_es[sep_es.field=='海洋']['title'].count()),
    ('Austria', sep_at[sep_at.field=='海洋']['title'].count()),
    ('Italy', sep_it[sep_it.field=='海洋']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='海洋']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='海洋']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='海洋']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='海洋']['title'].count())
]

chart = Map("2018年9月全球海洋领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_hy)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 15], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [128]:
chart = get_wc('海洋',nub=120,shapewc='circle')
chart

## 航空

2018年9月共**24条**动态

In [129]:
bar = get_field_src(sep_d, '航空')
bar

In [130]:
heatmap = get_field_date(sep_d, '航空', [1, 4])
heatmap

In [131]:
data_hk = [
    ('United States', sep_us[sep_us.field=='航空']['title'].count()),
    ('China', sep_cn[sep_cn.field=='航空']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='航空']['title'].count()),
    ('France', sep_fr[sep_fr.field=='航空']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='航空']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='航空']['title'].count()),
    ('Australia', sep_au[sep_au.field=='航空']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='航空']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='航空']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='航空']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='航空']['title'].count()),
    ('India', sep_id[sep_id.field=='航空']['title'].count()),
    ('Israel', sep_is[sep_is.field=='航空']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='航空']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='航空']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='航空']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='航空']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='航空']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='航空']['title'].count()),
    ('Spain', sep_es[sep_es.field=='航空']['title'].count()),
    ('Austria', sep_at[sep_at.field=='航空']['title'].count()),
    ('Italy', sep_it[sep_it.field=='航空']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='航空']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='航空']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='航空']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='航空']['title'].count())
]

chart = Map("2018年9月全球航空领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_hk)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 3], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [132]:
chart = get_wc('航空')
chart

## 航天

2018年9月共**23条**动态

In [133]:
bar = get_field_src(sep_d, '航天')
bar

In [134]:
heatmap = get_field_date(sep_d, '航天', [1, 4])
heatmap

In [135]:
data_ht = [
    ('United States', sep_us[sep_us.field=='航天']['title'].count()),
    ('China', sep_cn[sep_cn.field=='航天']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='航天']['title'].count()),
    ('France', sep_fr[sep_fr.field=='航天']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='航天']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='航天']['title'].count()),
    ('Australia', sep_au[sep_au.field=='航天']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='航天']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='航天']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='航天']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='航天']['title'].count()),
    ('India', sep_id[sep_id.field=='航天']['title'].count()),
    ('Israel', sep_is[sep_is.field=='航天']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='航天']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='航天']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='航天']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='航天']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='航天']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='航天']['title'].count()),
    ('Spain', sep_es[sep_es.field=='航天']['title'].count()),
    ('Austria', sep_at[sep_at.field=='航天']['title'].count()),
    ('Italy', sep_it[sep_it.field=='航天']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='航天']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='航天']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='航天']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='航天']['title'].count())
]

chart = Map("2018年9月全球航天领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_ht)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 3], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [136]:
chart = get_wc('航天', shapewc='pentagon')
chart

## 新材料

2018年9月共**24条**动态


In [137]:
bar = get_field_src(sep_d, '新材料')
bar

In [138]:
heatmap = get_field_date(sep_d, '新材料', [1, 4])
heatmap

In [139]:
data_cl = [
    ('United States', sep_us[sep_us.field=='新材料']['title'].count()),
    ('China', sep_cn[sep_cn.field=='新材料']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='新材料']['title'].count()),
    ('France', sep_fr[sep_fr.field=='新材料']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='新材料']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='新材料']['title'].count()),
    ('Australia', sep_au[sep_au.field=='新材料']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='新材料']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='新材料']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='新材料']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='新材料']['title'].count()),
    ('India', sep_id[sep_id.field=='新材料']['title'].count()),
    ('Israel', sep_is[sep_is.field=='新材料']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='新材料']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='新材料']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='新材料']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='新材料']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='新材料']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='新材料']['title'].count()),
    ('Spain', sep_es[sep_es.field=='新材料']['title'].count()),
    ('Austria', sep_at[sep_at.field=='新材料']['title'].count()),
    ('Italy', sep_it[sep_it.field=='新材料']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='新材料']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='新材料']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='新材料']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='新材料']['title'].count())
]

chart = Map("2018年9月全球新材料领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_cl)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 9], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False)
chart

In [140]:
chart = get_wc('新材料', shapewc='cardioid')
chart

## 先进制造

2018年9月共**23条**动态

# bar = get_field_src(sep_d, '先进制造')
bar

In [141]:
heatmap = get_field_date(sep_d, '先进制造', [1, 3])
heatmap

In [142]:
data_zz = [
    ('United States', sep_us[sep_us.field=='先进制造']['title'].count()),
    ('China', sep_cn[sep_cn.field=='先进制造']['title'].count()),
    ('United Kingdom', sep_uk[sep_uk.field=='先进制造']['title'].count()),
    ('France', sep_fr[sep_fr.field=='先进制造']['title'].count()),
    ('Germany', sep_ge[sep_ge.field=='先进制造']['title'].count()),
    ('Canada', sep_ca[sep_ca.field=='先进制造']['title'].count()),
    ('Australia', sep_au[sep_au.field=='先进制造']['title'].count()),
    ('Japan', sep_jp[sep_jp.field=='先进制造']['title'].count()),
    ('Korea', sep_kr[sep_kr.field=='先进制造']['title'].count()),
    ('Singapore', sep_sg[sep_sg.field=='先进制造']['title'].count()),
    ('Russia', sep_ru[sep_ru.field=='先进制造']['title'].count()),
    ('India', sep_id[sep_id.field=='先进制造']['title'].count()),
    ('Israel', sep_is[sep_is.field=='先进制造']['title'].count()),
    ('Switzerland', sep_ch[sep_ch.field=='先进制造']['title'].count()),
    ('Sweden', sep_se[sep_se.field=='先进制造']['title'].count()),
    ('Denmark', sep_dk[sep_dk.field=='先进制造']['title'].count()),
    ('Netherlands', sep_nl[sep_nl.field=='先进制造']['title'].count()),
    ('Finland', sep_fi[sep_fi.field=='先进制造']['title'].count()),
    ('Poland', sep_pl[sep_pl.field=='先进制造']['title'].count()),
    ('Spain', sep_es[sep_es.field=='先进制造']['title'].count()),
    ('Austria', sep_at[sep_at.field=='先进制造']['title'].count()),
    ('Italy', sep_it[sep_it.field=='先进制造']['title'].count()),
    ('Qatar', sep_qa[sep_qa.field=='先进制造']['title'].count()),
    ('Iran', sep_ir[sep_ir.field=='先进制造']['title'].count()),
    ('Saudi Arabia', sep_sa[sep_sa.field=='先进制造']['title'].count()),
    ('Brazil', sep_br[sep_br.field=='先进制造']['title'].count())
]

chart = Map("2018年9月全球先进制造领域活跃度地图", "data from 全球技术地图", **style.init_style)
attr, value = chart.cast(data_zz)
chart.add("科技前沿动态", attr, value, maptype="world", is_visualmap=True,visual_range_text=['不活跃', '活跃'],
              visual_range=[0, 5], visual_text_color='#fff',
              is_legend_show=False, is_map_symbol_show=False,)
chart

In [143]:
chart = get_wc('先进制造', shapewc='circle')
chart